# Library of LLM Content Analysis Tools

- uses the openrouter API with various access to a variety of models
- `classify_interaction()` is the main function, calls the `invoke_openrouter_ai()` function to call the API then extracts the data from the structured JSON response


In [ ]:
import os
import json
import requests 

def invoke_openrouter_ai(user_prompt, model = os.getenv("OPENROUTER_MODEL")):
    headers = {
        "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": user_prompt}
        ],
        #"response_format": "json",
        "temperature" : 0.0
    }
    #print(f"Model: {model}")
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
    return response.json()


In [ ]:
INTERACTION_PROMPT_TEMPLATE='''
    You are a research assistant specializing in qualatative analysis. 

    You are provided with transcript of a participant / AI interaction, which appears after the text TRANSCRIPT: 
    The research participant is a student enrolled in an introductory programming course, teaching Python. 
    The AI assistant responses are clearly labeled AI_ASSISTANT: The AI assistant was desiged to write code and always explain any code it writes.  

    You are looking for clear evidence of AI use by participants that is constructive to learning,
    meaning the student is using AI in ways to support their learning and skill development.

    CONSTRUCTIVE use cases are when participants engage with AI in ways which foster learning. 

    Examples:
    - Participant is specific in their questioning, or engages in dialog with AI to get to specifics.
    - Ask AI to explain their code 
    - Ask AI to clarify a programming concept, or provide examples of the concept
    - Ask AI to debug a problem in their code

    UNCONSTRUTIVE use cases involve students cognitively offloading their learning to the AI. 

    Examples:
    - participant asks general questions without any form of context
    - Ask AI for solutions to assignments
    - Ask AI to turn an algorithm into code
    - Ask AI to solve the problem for them


    Classify the interation as:

    - CONSTRUCTIVE, UNCONSTRUTIVE or INCONCLUSIVE
    - Provide brief justification for the classification, along with evidence from the transcript.


    Important things to note:

    - Always output in JSON format, based on the examples.
    - Some participants paste the assignment instructions into the AI as the start of their transcript. do not include this text in your classification.
    - Focus on the participant questions and responses to the AI assistant. The AI assistant is designed to respond similarly to each participant, so their responses should not be included in your analysis.


    Output:

    ALWAYS Generate output in JSON format, here are some examples of the JSON structure:

    Example 1:
    ```JSON
    {
        "classification": "CONSTRUCTIVE",
        "justification": {
            "text" : "The participant engages with the AI by asking specific questions and seeking clarification on programming concepts, which fosters their learning.",
            "evidence": [
                "Participant asks 'can you explain this code?'",
                "Participant requests examples of how to use the 'for' loop."
            ]
        }
    }
    ```

    Example 2:
    ```JSON
    {
        "classification": "UNCONSTRUCTIVE",
        "justification": {
            "text" : "The participant asks the AI to complete their work for them, which does not support their learning.",
            "evidence": [
                "Participant asks 'Can you complete Assignment 3 for me?"
            ]
        }
    }
    ```

    TRANSCRIPT:
    {transcript}

    '''


In [12]:
RESPONSE_FORMAT = '''
    {
    "$schema": "http://json-schema.org/draft-04/schema#",
    "type": "object",
    "properties": {
        "classification": {
        "type": "string"
        },
        "justification": {
        "type": "object",
        "properties": {
            "behavior_code": {
            "type": "string"
            },
            "rationale": {
            "type": "string"
            },
            "evidence": {
            "type": "array",
            "items": [
                {
                "type": "string"
                },
                {
                "type": "string"
                }
            ]
            }
        },
        "required": [
            "behavior_code",
            "rationale",
            "evidence"
        ]
        }
    },
    "required": [
        "classification",
        "justification"
    ]
    }

'''

In [13]:
INTERACTION_PROMPT_TEMPLATEv2='''
    You are a research assistant specializing in qualitative analysis. You are highly skilled at coding qualitative data.

    BACKGROUND:

    You are provided with transcript of a participant / AI interaction, which appears after the text TRANSCRIPT: 
    The research participant is a student enrolled in an introductory programming course, teaching Python. 
    The AI assistant responses are clearly labeled AI_ASSISTANT: The AI assistant was desiged to write code and always explain any code it writes.  

    Your are looking for clear evidence of AI use by participants that is constructive to learning,
    meaning the student is using AI in ways to support their learning and skill development as opposed to
    deferring their learning to the AI.

    Your task is to code each transcript according to the following codebook:

    | Behavior (Code)      | Definition                                                                                           | Examples<br>(Taken From Examples)                                                                                                                                                    | Classification |
    | -------------------- | ---------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | -------------- |
    | Seeking Answers      | Participant makes a direct ask for AI to complete an assignment or lab.                              | "Can you code solution to assignment ABC?"<br>"How do I complete section XYZ on the lab?"                                                                                            | Unproductive   |
    | Cognitive Offloading | Participant asks AI to do their work for them. Thus a deep connection to the material is not formed. | "Can you convert  this algorithm into code?"<br>"Can you provide an approach to start this assignment?"<br>"Fix this error for me."                                                  | Unproductive   |
    | Debugging            | Participant asks the AI to help fix  errors, with emphasis on understanding the error.               | "What does this error mean?"<br>"Can you help me locate the error in my code?"<br>"Can you help me troubleshoot my code?"                                                            | Productive     |
    | Understanding        | Asking the AI to explain a concept or provide code examples of a concept.                            | Can you provide an example of a nested loop?<br>Why would I use a nested IF verses elif?<br>Can you help me understand dictionaries?<br>When would I use a list versus a dictionary? | Productive     |
    | Meta                 | Participant asks questions about the AI's capabilities.                                              | Do you know about seaborn?<br>How much stuff can you remember from our converations?                                                                                                 | Inconclusive   |
    | Off-Topic            | Not related to subject. Questions not related to programming.                                        | "When is the exam?"<br>"Can you tell me the weather?"                                                                                                                                | Inconclusive   |
    

    RULES TO FOLLOW:

    - Always output in JSON format, based on the schema in the JSON SCHEMA OUTPUT section.
    - Some participants paste the assignment instructions into the AI as the start of their transcript. do not include this text in your classification.
    - Focus on the participant questions and responses to the AI assistant. The AI assistant is designed to respond similarly to each participant, so their responses should not be included in your analysis.
    - When the interactions are mixed, classify the dominant theme or behavior exhibited by the participant.


    JSON SCHEMA OUTPUT:

    ```JSON
    {
    "$schema": "http://json-schema.org/draft-04/schema#",
    "type": "object",
    "properties": {
        "classification": {
        "type": "string"
        },
        "justification": {
        "type": "object",
        "properties": {
            "behavior_code": {
            "type": "string"
            },
            "rationale": {
            "type": "string",
            "description": "Your rationale for the classification. Why did you choose the behavior_code based on the evidence?"
            },
            "evidence": {
            "description": "Evidence supporting the classification.",
            "type": "array",
            "items": [
                {
                "type": "string"
                },
                {
                "type": "string"
                }
            ]
            }
        },
        "required": [
            "behavior_code",
            "rationale",
            "evidence"
        ]
        }
    },
    "required": [
        "classification",
        "justification"
    ]
    }
    ```

    EXAMPLE OUTPUT:

    Examples of Understanding
    ```json
        {
            "classification": "CONSTRUCTIVE",
            "justification": {
                "behavior_code" : "Understanding",
                "rationale" : "The participant engages with the AI by asking specific questions and seeking clarification on programming concepts, which fosters their learning.",
                "evidence": [
                    "Participant asks 'can you explain this code?'",
                    "Participant requests examples of how to use the 'for' loop.",
                    "Participant asks 'When should I use int versus float?'",
                    "Participant asks 'Explain string interpolation.'",
                    "Participant asks for practice questions in a specific context: 'Can you provide some practice exam questions using elif and the for loop?'"
                ]
            }
        }
    ```
    Example of Seeking Answers
    ```json
    {
        "classification": "UNCONSTRUCTIVE",
        "justification": {
            "behavior_code" : "Seeking Answers",
            "rationale" : "The participant is looking for answers on the homework and lab assignments. There an emphasis on task-completion versus learning.",
            "evidence": [
                "Participant asks 'Can you write the code for Assignment 3?'",
                "Participant asks 'How do I complete section 1.4 from the lab?",
            ]
        }
    }


    Example of Debugging
    ```JSON
    {
        "classification": "CONSTRUCTIVE",
        "justification": {
            "behavior_code" : "Debugging",
            "rationale" : "The participant is actively debugging their code by asking specific questions about error messages and seeking clarification on programming concepts.",
            "evidence": [
                "Participant includes code then asks 'What does this error message mean?'",
                "Participant includes code asks 'How can I fix this bug in my code?'",
                "Participant includes code and sample output then asks 'Why is my program not producing the expected output?'",
                "Participant includes code then asks 'I have an error on line 15 can you help me understand it?'"
            ]
        }
    }

    Example of Cognitive Offloading
    ```JSON
    {
        "classification": "UNCONSTRUCTIVE",
        "justification": {
            "behavior_code" : "Cognitive Offloading",
            "rationale" : "The participant is relying on the AI to provide solutions without attempting to solve the problems themselves. This includes asking errors to be fixed without an effort to understand the underlying concepts.",
            "evidence": [
                "Participant includes an algorithm then asks 'Can you write the code for me?'",
                "Participant includes instructions then asks 'How do I complete this assignment?'",
                "Participant includes code then asks 'Fix my error'",
                "Participant pastes code without context."
                "Participant asks for example exam questions without providing context. (e.g. 'Can you give me some practice questions?')"
            ]
        }
    }

    Example of Meta
    ```json
    {
        "classification": "INCONCLUSIVE",
        "justification": {
            "behavior_code" : "Meta",
            "rationale" : "The participant is asking about the AI capabilities.",
            "evidence": [
                "Participant asks 'What do you know about python?'",
                "Participant asks 'How much can you remember from our conversations?'",
                "Participant asks 'Do you know about seaborn?'"
            ]
        }
    }

    Example of Off-Topic
    ```json
    {
        "classification": "INCONCLUSIVE",
        "justification": {
            "behavior_code" : "Off-Topic",
            "rationale" : "The participant is asking questions that are not relevant to the course topics. The questions are not about python programming.",
            "evidence": [
                "Participant asks 'What is the weather like today?'",
                "Participant asks 'Can you recommend a good restaurant?'",
                "Participant asks 'When is the homework due?'"
            ]
        }
    }

    TRANSCRIPT:
    {transcript}

    '''




In [14]:
INTERACTION_PROMPT_TEMPLATEv3='''   
    You are a PhD research assistant who is highly skilled at coding qualitative data for categorical content analysis.

    As part of a research study, participants in an introductory programming course were given access to an AI chatbot knowledgeable in the python programming language.
    Your task is to analyze transcripts of interactions between the participants and the AI chatbot, and classify the the participant behavior based on a predefined codebook.
    The codebook can be found within the <CODEBOOK> XML tag. 
    The coding rules should be followed. These are in the <RULES> XML tag.

    Transcript:

    - The transcript is a conversation between the participant and the AI, located within the <TRANSCRIPT> XML tag.
    - Participant text starts with PARTICIPANT_#: where # is the participant number.
    - The AI assistant responses are clearly labeled AI_ASSISTANT: The AI assistant was desiged to write code and always explain any code it writes.
  
    <CODEBOOK>

    Your goal is to review the transcript and classify the participant's behavior according to the following codebook, keeping in mind the context of an introductory programming course and the objective of fostering learning and skill development among participants.

    | Behavior (Code)      | Classification  | Definition                                                                                             | Examples<br>(Taken From Examples)                                                                                                                                                    | 
    | -------------------- | --------------- | ------------------------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | 
    | Seeking Answers      | Task Completion | Participant makes a direct ask for AI to complete an assignment or lab. There is no conversation.      | "Can you code solution to assignment ABC?"<br>"How do I complete section XYZ on the lab?"                                                                                            | 
    | Overreliance         | Task Completion | Participant asks AI to do their work for them. Thus a deep connection to the material is not formed.   | "Can you convert this algorithm into code?"<br>"Can you provide an approach to start this assignment?"<br>"Fix this error for me."                                                  | 
    | Scaffolding          | Learning        | Participant engages with the AI in a manner that forsters their learning or understanding.             | "What does this error mean?"<br>"Can you help me locate the error in my code?"<br>"Can you help me troubleshoot my code?"                                                            | 
    | Questioning          | Learning        | Participant asks the AI to explain a concept or provide examples of a concept to better understand it. | Can you provide an example of a nested loop?<br>Why would I use a nested IF verses elif?<br>Can you help me understand dictionaries?<br>When would I use a list versus a dictionary? | 
    | Unable to determine  | Inconclusive    | The transcript does not provide enough information to classify the participant's behavior. Off topic question not about the subject matter.  | "When is the exam?"<br>"Can you tell me the weather?"<br>What did I ask you last week?                                                       |

    </CODEBOOK>

    <RULES>

    - Some participants paste the assignment instructions into the AI as the start of their transcript. do not include this text in your classification.
    - When the interactions are mixed, classify the dominant theme or behavior exhibited by the participant.
    - Always output in JSON format, based on the schema in the JSON SCHEMA OUTPUT section. 
      You must include a classification (Task Completion, Learning, Inconclusive), behavior_code (Seeking Answers, Overreliance, Scaffolding, Learning, Unable to determine), 
      a rationale (justify your coding) and evidence (what in the transcript supports your ).

    JSON SCHEMA OUTPUT:

    ```JSON
    {
    "$schema": "http://json-schema.org/draft-04/schema#",
    "type": "object",
    "properties": {
        "classification": {
        "type": "string"
        "description": "Overall classification of the participant behavior: Task Completion, Learning, Inconclusive"
        },
        "justification": {
        "type": "object",
        "properties": {
            "behavior_code": {
            "type": "string"
            "description": "The specific behavior code from the codebook that best describes the participant's behavior: Seeking Answers, Overreliance, Scaffolding, Questioning, Unable to determine"
            },
            "rationale": {
            "type": "string",
            "description": "Your rationale for the classification. Why did you choose the behavior_code based on the evidence?"
            },
            "evidence": {
            "description": "Evidence supporting the classification.",
            "type": "array",
            "items": [
                {
                "type": "string"
                "description": "Specific evidence from the transcript supporting the classification. Direct quotes from the participant."
                },
                {
                "type": "string"
                "description": "Specific evidence from the transcript supporting the classification. Direct quotes from the participant."
                }
            ]
            }
        },
        "required": [
            "behavior_code",
            "rationale",
            "evidence"
        ]
        }
    },
    "required": [
        "classification",
        "justification"
    ]
    }
    ```

    EXAMPLE OUTPUT:

    Examples of "Questioning"
    ```json
        {
            "classification": "Learning",
            "justification": {
                "behavior_code" : "Questioning",
                "rationale" : "The participant engages with the AI by asking specific questions and seeking clarification on programming concepts, which fosters their learning.",
                "evidence": [
                    "Participant asks 'can you explain this code?'",
                    "Participant requests examples of how to use the 'for' loop.",
                    "Participant asks 'When should I use int versus float?'",
                    "Participant asks 'Explain string interpolation.'",
                    "Participant asks for practice questions in a specific context: 'Can you provide some practice exam questions using elif and the for loop?'"
                    "Participant pastes in code and asks for an explanation of why its not working."
                ]
            }
        }
    ```
    Example of "Seeking Answers"
    ```json
    {
        "classification": "Task Completion",
        "justification": {
            "behavior_code" : "Seeking Answers",
            "rationale" : "The participant is looking for answers on the homework and lab assignments. There an emphasis on task-completion versus learning.",
            "evidence": [
                "Participant asks 'Can you write the code for Assignment 3?'",
                "Participant asks 'How do I complete section 1.4 from the lab?",
                "Participant pastes in code, expecting the AI to fix it without any effort to understand the underlying concepts."
            ]
        }
    }


    Example of Scaffolding
    ```JSON
    {
        "classification": "Learning",
        "justification": {
            "behavior_code" : "Scaffolding",
            "rationale" : "The participant is actively debugging their code by asking specific questions about error messages and seeking clarification on programming concepts.",
            "evidence": [
                "Participant includes code then asks 'What does this error message mean?'",
                "Participant includes code asks 'How can I fix this bug in my code?'",
                "Participant includes code and sample output then asks 'Why is my program not producing the expected output?'",
                "Participant includes code then asks 'I have an error on line 15 can you help me understand it?'"
            ]
        }
    }

    Example of Cognitive Offloading
    ```JSON
    {
        "classification": "Task Completion",
        "justification": {
            "behavior_code" : "Overreliance",
            "rationale" : "The participant is relying on the AI to provide solutions without attempting to solve the problems themselves. This includes asking errors to be fixed without an effort to understand the underlying concepts.",
            "evidence": [
                "Participant includes an algorithm then asks 'Can you write the code for me?'",
                "Participant includes instructions then asks 'How do I complete this assignment?'",
                "Participant includes code then asks 'Fix my error'",
                "Participant pastes code without context."
                "Participant asks for example exam questions without providing context. (e.g. 'Can you give me some practice questions?')"
            ]
        }
    }

    Example of Inconclusive
    ```json
    {
        "classification": "Inconclusive",
        "justification": {
            "behavior_code" : "Unable to determine",
            "rationale" : "The participant is asking about the AI capabilities.",
            "evidence": [
                "Participant asks 'What do you know about python?'",
                "Participant asks 'How much can you remember from our conversations?'",
                "Participant asks 'Do you know about seaborn?'"
            ]
        }
    }

    Example of Inconclusive
    ```json
    {
        "classification": "Inconclusive",
        "justification": {
            "behavior_code" : "Unable to determine",
            "rationale" : "The participant is asking questions that are not relevant to the course topics. The questions are not about python programming.",
            "evidence": [
                "Participant asks 'What is the weather like today?'",
                "Participant asks 'Can you recommend a good restaurant?'",
                "Participant asks 'When is the homework due?'",
                "If Python better than the R programming language?"
            ]
        }
    }

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
'''



In [ ]:
def classify_interaction(template, transcript_text, model, retries=3):

    prompt = template.replace("{transcript}", transcript_text, )
    attempts = 1
    

    while attempts <= retries:
        try:
            error = ""
            remaining = retries - attempts
            response = invoke_openrouter_ai(prompt, model)
            ai_interaction_text = response['choices'][0]['message']['content'].strip().replace("```json", "").replace("```JSON", "").replace("```", "").strip()
            if not ai_interaction_text.startswith("{"):
                # strip off text until {
                ai_interaction_text = ai_interaction_text[ai_interaction_text.index("{"):]
            
            # attempt to convert to json
            ai_interaction = json.loads(ai_interaction_text)
            break

        except json.JSONDecodeError as e:
            error = f"Error decoding JSON from AI response. text={ai_interaction_text}, error={e}, remaining={remaining}"
            print(error)
        except KeyError as e:
            error = f"Error retrieving content from AI response. response={response}, error={e}, remaining={remaining}"
            print(error)
        except Exception as e:
            error = f"Error invoking AI API. prompt={prompt}, error={e}, remaining={remaining}"
        finally:
            if remaining ==0:
                return    {
                    "classification": "Inconclusive",
                    "justification": {
                        "behavior_code" : "Meta",
                        "rationale" : "Unable to parse the AI output to JSON.",
                        "evidence": [
                            error
                        ]
                    }
                }
            attempts += 1

    return ai_interaction

In [16]:
def read_transcript(file_name):
    file_path = f"../data/processed/transcripts/{file_name}"
    with open(file_path, 'r') as file:
        return file.read()

In [18]:
# transcript = "Participant_87_session_ff79dbdd-3e5e-4adf-8b68-751694038039_transcript.txt"
# transcript_text = read_transcript(transcript)


# i1 = classify_interaction(INTERACTION_PROMPT_TEMPLATE, transcript_text, os.environ.get("OPENROUTER_MODEL"))
# i2 = classify_interaction(INTERACTION_PROMPT_TEMPLATEv2, transcript_text, os.environ.get("OPENROUTER_MODEL"))
# i3 = classify_interaction(INTERACTION_PROMPT_TEMPLATEv3, transcript_text, os.environ.get("OPENROUTER_MODEL"))
# print(i1)
# print(i2)
# print(i3)


{'classification': 'UNCONSTRUCTIVE', 'justification': {'text': 'The participant simply pasted their code without asking any specific questions or requesting explanations. They did not engage in any dialogue to understand the concepts or seek clarification on specific issues. This represents cognitive offloading where the AI identifies and fixes all problems without the student actively participating in the learning process.', 'evidence': ['Participant only provided code without any questions or context', "No follow-up questions or engagement with the AI's explanation", 'No attempt to understand specific concepts or debug particular issues']}}
{'classification': 'UNCONSTRUCTIVE', 'justification': {'behavior_code': 'Cognitive Offloading', 'rationale': 'The participant simply pasted their code without any specific question, context, or attempt to understand the errors. They did not ask what the errors meant or request help understanding the concepts. This represents cognitive offloading a

In [23]:
# MODELS_TO_TEST = ["x-ai/grok-4-fast","openai/gpt-5-mini", "google/gemini-2.5-flash", "anthropic/claude-sonnet-4"]

# for model in MODELS_TO_TEST:
#     print(f"\nTesting model: {model}")
#     #i1 = classify_interaction(INTERACTION_PROMPT_TEMPLATE, transcript_text, model)
#     i2 = classify_interaction(INTERACTION_PROMPT_TEMPLATEv3, transcript_text, model)
#     print(i2)


Testing model: x-ai/grok-4-fast
{'classification': 'Task Completion', 'justification': {'behavior_code': 'Overreliance', 'rationale': 'The participant pasted buggy code without any explicit question or request for explanation, relying on the AI to identify issues and provide a complete corrected version. This indicates a lack of engagement with the material, as the participant is offloading the debugging and correction process to the AI without seeking to understand the errors.', 'evidence': ['Participant provides incomplete code: \'line_count = 0 filename = "mbox-tiny.txt" with open(filename, \'r\') for line in f.readlines(): line_count = 1 print("there are {line_count} lines in the file")\'', 'No questions or requests for clarification from the participant; the code has clear errors (missing colon, no file object assignment, incorrect line counting) but is presented without context or inquiry.']}}

Testing model: openai/gpt-5-mini
{'classification': 'Learning', 'justification': {'be

In [4]:
if False:
    invoke_openrouter_ai("Hello, can you help me write some python code?", model="openai/gpt-4")